Многослойный персептрон
- Реализовать архитектуру многослойного персептрона. 
- Реализовать алгоритм обратного распространения ошибки для обучения сети;
- Обучить на очищенном датасете  из 1-ой лабораторной. 
- Реализовать перцептрон - регрессор; 
- Обработать датасет с классификацией грибов по алгоритму из 1-й лабораторной работы.
- Обучить перцептрон на датасете с классификацией грибов. 
- Реализовать классификатор.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch 
import torch.nn as nn
import sklearn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from ucimlrepo import fetch_ucirepo
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder 
from sklearn.neural_network import MLPClassifier, MLPRegressor

# Перцептрон - Регрессор (ноутбуки)

In [2]:
ldf = pd.read_csv("./Laptop_price.csv")

In [3]:
ldf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Brand             1000 non-null   object 
 1   Processor_Speed   1000 non-null   float64
 2   RAM_Size          1000 non-null   int64  
 3   Storage_Capacity  1000 non-null   int64  
 4   Screen_Size       1000 non-null   float64
 5   Weight            1000 non-null   float64
 6   Price             1000 non-null   float64
dtypes: float64(4), int64(2), object(1)
memory usage: 54.8+ KB


In [4]:
ldf['Brand'].unique()

array(['Asus', 'Acer', 'Lenovo', 'HP', 'Dell'], dtype=object)

In [5]:
ldf = pd.get_dummies(ldf,columns=['Brand'])

In [6]:
ldf.head()

,Processor_Speed,RAM_Size,Storage_Capacity,Screen_Size,Weight,Price,Brand_Acer,Brand_Asus,Brand_Dell,Brand_HP,Brand_Lenovo
0,3.830296,16,512,11.185147,2.641094,17395.093065,0,1,0,0,0
1,2.912833,4,1000,11.311372,3.260012,31607.605919,1,0,0,0,0
2,3.241627,4,256,11.853023,2.029061,9291.023542,0,0,0,0,1
3,3.806248,16,512,12.280360,4.573865,17436.728334,1,0,0,0,0
4,3.268097,32,1000,14.990877,4.193472,32917.990718,1,0,0,0,0


In [7]:
X = ldf.drop(columns=['Price']).values
y = ldf['Price'].values.reshape(-1, 1)


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
#scaler = StandardScaler()
#X_train = scaler.fit_transform(X_train)
#X_test = scaler.transform(X_test)


In [10]:
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

## sklearn model 

In [11]:
model = MLPRegressor(hidden_layer_sizes=(10,10,10))

In [12]:
model.fit(X_train,y_train)

/usr/lib64/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/lib64/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor(hidden_layer_sizes=(10, 10, 10))

In [13]:
model.score(X=X_test,y=y_test)

0.9953021036373197

In [14]:
model.get_params()

{'activation': 'relu',
 'alpha': 0.0001,
 'batch_size': 'auto',
 'beta_1': 0.9,
 'beta_2': 0.999,
 'early_stopping': False,
 'epsilon': 1e-08,
 'hidden_layer_sizes': (10, 10, 10),
 'learning_rate': 'constant',
 'learning_rate_init': 0.001,
 'max_fun': 15000,
 'max_iter': 200,
 'momentum': 0.9,
 'n_iter_no_change': 10,
 'nesterovs_momentum': True,
 'power_t': 0.5,
 'random_state': None,
 'shuffle': True,
 'solver': 'adam',
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': False,
 'warm_start': False}

## Своя модель

### Код

In [15]:
# Линейный слой
class Linear:
  def __init__(self, input_size=None, output_size=None):
    self.X = None
    self.input_size = input_size
    self.output_size = output_size
    self.W = np.random.randn(input_size, output_size)
    self.bias = np.zeros((1, output_size))

  def __call__(self, X):
    self.X = X
    result = np.dot(X, self.W) + self.bias
    return result

  def prop(self, grad_prev, learning_rate):
    grad_X = np.dot(grad_prev, self.W.T)
    grad_W = np.dot(self.X.T, grad_prev)
    grad_bias = np.sum(grad_prev, axis=0)

    self.W -= learning_rate * grad_W
    self.bias -= learning_rate * grad_bias
    return grad_X



In [16]:
# Функция активации
class Sigmoid:
  def __init__(self):
    self.y = None

  def __call__(self, x):
    self.y = self.sigmoid(x)
    return self.y


  def sigmoid(self,x):
      return 1. / (1. + np.exp(-x))
    
  def sigmoid_deriv(self, x):
      return x * (1 - x)
    
  def prop(self, grad_prev, learning_rate):
    return grad_prev * self.sigmoid_deriv(self.y)

In [17]:
# Перцептрон 
class MLP:
  def __init__(self, layers, max_iter=100, batch_size=100, learning_rate=0.001):
    self.layers = layers
    self.max_iter = max_iter
    self.batch_size = batch_size
    self.learning_rate = learning_rate

  @staticmethod
  def score(y_true, y_pred):
    if np.sum((y_true - np.mean(y_true)) ** 2) == 0:
      return 1
    return 1 - (np.sum((y_true - y_pred) ** 2) / np.sum((y_true - np.mean(y_true)) ** 2))


  def predict(self, input_data):
    prev_data = input_data
    for layer in self.layers:
      prev_data = layer(prev_data)
    return prev_data

  def _prop(self, grad_prev, learning_rate):
    for layer in reversed(self.layers):
      grad_prev = layer.prop(grad_prev, learning_rate)

  def fit(self, X, y):
    for iter in range(self.max_iter):
      permutation = np.random.permutation(len(X))
      X_shuffled = X[permutation]
      y_shuffled = y[permutation]

      total_loss = 0.
      for batch_first in range(0, len(X), self.batch_size):
        batch_last = min(batch_first + self.batch_size, len(X))

        X_batch = X_shuffled[batch_first:batch_last]
        y_batch = y_shuffled[batch_first:batch_last]

        y_pred = self.predict(X_batch)
        total_loss += self._loss(y_batch, y_pred)
        grad_prev = self._grad_loss(y_batch, y_pred)
        self._prop(grad_prev, self.learning_rate)

      print(f'iteration: {iter}, loss: {total_loss}')

  @staticmethod
  def _loss(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

  @staticmethod
  def _grad_loss(y_true, y_pred):
    return 2 * (y_pred - y_true) / len(y_true)

In [18]:
class Normalizer:
  def __init__(self):
    self.mean = None
    self.std = None

  def fit_transform(self, data):
    self.mean = np.mean(data, axis=0)
    self.std = np.std(data, axis=0)
    return (data - self.mean) / self.std


### Обучение

In [19]:
df = pd.read_csv('Laptop_price.csv')
print(df.head())

X = df[['Storage_Capacity', 'RAM_Size', 'Processor_Speed', 'Screen_Size', 'Weight']].values
y = df['Price'].values.reshape(-1, 1)

normalizerX = Normalizer()
normalizerY = Normalizer()

X = normalizerX.fit_transform(X)
y = normalizerY.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


    Brand  Processor_Speed  RAM_Size  Storage_Capacity  Screen_Size    Weight  \
0    Asus         3.830296        16               512    11.185147  2.641094   
1    Acer         2.912833         4              1000    11.311372  3.260012   
2  Lenovo         3.241627         4               256    11.853023  2.029061   
3    Acer         3.806248        16               512    12.280360  4.573865   
4    Acer         3.268097        32              1000    14.990877  4.193472   

          Price  
0  17395.093065  
1  31607.605919  
2   9291.023542  
3  17436.728334  
4  32917.990718  


In [20]:

model = MLP([
    Linear(X_train.shape[1], 10),
    Sigmoid(),
    Linear(10, 30),
    Sigmoid(),
    Linear(30, 1)
], learning_rate=0.001, max_iter=1000)
model.fit(X_train, y_train)

iteration: 0, loss: 13.272036227059246
iteration: 1, loss: 11.470309664364787
iteration: 2, loss: 10.196378574907325
iteration: 3, loss: 9.286482918506685
iteration: 4, loss: 8.623188728652629
iteration: 5, loss: 8.129667123493846
iteration: 6, loss: 7.756932509485049
iteration: 7, loss: 7.463859523608862
iteration: 8, loss: 7.227065382565684
iteration: 9, loss: 7.02929083635966
iteration: 10, loss: 6.858880552021258
iteration: 11, loss: 6.70746626265999
iteration: 12, loss: 6.571101726612962
iteration: 13, loss: 6.4431852249450134
iteration: 14, loss: 6.323746880532601
iteration: 15, loss: 6.211078057610908
iteration: 16, loss: 6.10024261581637
iteration: 17, loss: 5.9961851164740825
iteration: 18, loss: 5.893663854321205
iteration: 19, loss: 5.794046889007545
iteration: 20, loss: 5.696960128492919
iteration: 21, loss: 5.60198574025409
iteration: 22, loss: 5.509413428031145
iteration: 23, loss: 5.420125764302803
iteration: 24, loss: 5.331002732045797
iteration: 25, loss: 5.24449306430

In [21]:
y_exp = y_test.T[0]
y_pred = model.predict(X_test).T[0]
model.score(y_exp,y_pred)

0.9543791425248009

# Перцептрон - Классификатор (грибы)

In [22]:
gdf = pd.read_csv("./mushroom/expanded.csv")

In [23]:
gdf.replace("?", pd.NA, inplace=True)

In [24]:
nans = gdf.isna().sum()
print(nans[nans != 0])

stalk-root    2480
dtype: int64


In [25]:
gdf['stalk-root'].fillna(gdf['stalk-root'].mode()[0], inplace=True)

In [26]:
nans = gdf.isna().sum()
print(nans[nans != 0])

Series([], dtype: int64)


In [27]:
u =gdf.apply(lambda x: x.unique())
u

class                                                     [EDIBLE, POISONOUS]
cap-shape                      [CONVEX, FLAT, BELL, SUNKEN, KNOBBED, CONICAL]
cap-surface                                 [SMOOTH, FIBROUS, SCALY, GROOVES]
cap-color                   [WHITE, YELLOW, BROWN, GRAY, RED, PINK, PURPLE...
brusies                                                         [BRUISES, NO]
odor                        [ALMOND, ANISE, NONE, PUNGENT, CREOSOTE, FOUL,...
gill-attachment                                              [FREE, ATTACHED]
gill-spacing                                                 [CROWDED, CLOSE]
gill-size                                                     [NARROW, BROAD]
gill-color                  [WHITE, PINK, BROWN, GRAY, BLACK, CHOCOLATE, P...
stalk-shape                                             [TAPERING, ENLARGING]
stalk-root                                     [BULBOUS, CLUB, ROOTED, EQUAL]
stalk-surface-above-ring                      [SMOOTH, FIBROUS, 

In [28]:
X = gdf.drop(columns=['class'])
y = gdf['class']

In [29]:
cat_features = [x for x in X.columns]
ohe_features = [x for x in X.columns]

In [30]:
for col in cat_features:
    if (len(X[col].unique()) == 2):
        X[col] = (X[col] == X[col][0]).astype(int)
        ohe_features.remove(col)
        

In [31]:
ring_mapping = {'NONE':0, 'ONE': 1, 'TWO': 2}
X['ring-number'] = X['ring-number'].map(ring_mapping)
ohe_features.remove('ring-number')


In [32]:
pd.set_option('display.max_columns', None)
X.sample(n=20)

,cap-shape,cap-surface,cap-color,brusies,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
1481,CONVEX,FIBROUS,GRAY,0,NONE,1,1,0,PINK,1,EQUAL,FIBROUS,SMOOTH,WHITE,WHITE,PARTIAL,WHITE,1,EVANESCENT,BLACK,ABUNDANT,GRASSES
965,CONVEX,SCALY,WHITE,1,PUNGENT,1,0,1,WHITE,0,EQUAL,SMOOTH,SMOOTH,WHITE,WHITE,PARTIAL,WHITE,1,PENDANT,BROWN,SCATTERED,GRASSES
6775,FLAT,SMOOTH,RED,0,FISHY,1,0,1,BUFF,1,BULBOUS,SILKY,SILKY,PINK,PINK,PARTIAL,WHITE,1,EVANESCENT,WHITE,SEVERAL,WOODS
4264,CONVEX,FIBROUS,YELLOW,0,FOUL,1,0,0,GRAY,0,BULBOUS,SILKY,SILKY,BUFF,PINK,PARTIAL,WHITE,1,LARGE,CHOCOLATE,SOLITARY,GRASSES
90,FLAT,FIBROUS,YELLOW,1,ANISE,1,1,1,WHITE,1,BULBOUS,SMOOTH,SMOOTH,WHITE,WHITE,PARTIAL,WHITE,1,PENDANT,PURPLE,SEVERAL,WOODS
7114,FLAT,SCALY,BROWN,0,FOUL,1,0,1,BUFF,1,BULBOUS,SMOOTH,SMOOTH,WHITE,WHITE,PARTIAL,WHITE,1,EVANESCENT,WHITE,SEVERAL,WOODS
1341,CONVEX,SMOOTH,GRAY,0,NONE,1,1,0,BROWN,1,EQUAL,FIBROUS,FIBROUS,WHITE,WHITE,PARTIAL,WHITE,1,EVANESCENT,BLACK,ABUNDANT,GRASSES
4044,CONVEX,FIBROUS,PINK,0,CREOSOTE,1,0,1,BROWN,0,BULBOUS,SMOOTH,SMOOTH,WHITE,WHITE,PARTIAL,WHITE,1,PENDANT,BLACK,SCATTERED,WOODS
443,CONVEX,SMOOTH,YELLOW,1,ALMOND,1,0,0,BROWN,0,CLUB,SMOOTH,SMOOTH,WHITE,WHITE,PARTIAL,WHITE,1,PENDANT,BLACK,NUMEROUS,GRASSES
728,FLAT,SCALY,YELLOW,1,ANISE,1,0,0,WHITE,0,ROOTED,SMOOTH,SCALY,WHITE,WHITE,PARTIAL,WHITE,1,PENDANT,BLACK,SOLITARY,GRASSES


In [33]:
X = pd.get_dummies(X, columns=ohe_features)

In [34]:
print(X.shape)
X.sample(n=5)


(8416, 109)


,brusies,gill-attachment,gill-spacing,gill-size,stalk-shape,ring-number,cap-shape_BELL,cap-shape_CONICAL,cap-shape_CONVEX,cap-shape_FLAT,cap-shape_KNOBBED,cap-shape_SUNKEN,cap-surface_FIBROUS,cap-surface_GROOVES,cap-surface_SCALY,cap-surface_SMOOTH,cap-color_BROWN,cap-color_BUFF,cap-color_CINNAMON,cap-color_GRAY,cap-color_GREEN,cap-color_PINK,cap-color_PURPLE,cap-color_RED,cap-color_WHITE,cap-color_YELLOW,odor_ALMOND,odor_ANISE,odor_CREOSOTE,odor_FISHY,odor_FOUL,odor_MUSTY,odor_NONE,odor_PUNGENT,odor_SPICY,gill-color_BLACK,gill-color_BROWN,gill-color_BUFF,gill-color_CHOCOLATE,gill-color_GRAY,gill-color_GREEN,gill-color_ORANGE,gill-color_PINK,gill-color_PURPLE,gill-color_RED,gill-color_WHITE,gill-color_YELLOW,stalk-root_BULBOUS,stalk-root_CLUB,stalk-root_EQUAL,stalk-root_ROOTED,stalk-surface-above-ring_FIBROUS,stalk-surface-above-ring_SCALY,stalk-surface-above-ring_SILKY,stalk-surface-above-ring_SMOOTH,stalk-surface-below-ring_FIBROUS,stalk-surface-below-ring_SCALY,stalk-surface-below-ring_SILKY,stalk-surface-below-ring_SMOOTH,stalk-color-above-ring_BROWN,stalk-color-above-ring_BUFF,stalk-color-above-ring_CINNAMON,stalk-color-above-ring_GRAY,stalk-color-above-ring_ORANGE,stalk-color-above-ring_PINK,stalk-color-above-ring_RED,stalk-color-above-ring_WHITE,stalk-color-above-ring_YELLOW,stalk-color-below-ring_BROWN,stalk-color-below-ring_BUFF,stalk-color-below-ring_CINNAMON,stalk-color-below-ring_GRAY,stalk-color-below-ring_ORANGE,stalk-color-below-ring_PINK,stalk-color-below-ring_RED,stalk-color-below-ring_WHITE,stalk-color-below-ring_YELLOW,veil-type_PARTIAL,veil-color_BROWN,veil-color_ORANGE,veil-color_WHITE,veil-color_YELLOW,ring-type_EVANESCENT,ring-type_FLARING,ring-type_LARGE,ring-type_NONE,ring-type_PENDANT,spore-print-color_BLACK,spore-print-color_BROWN,spore-print-color_BUFF,spore-print-color_CHOCOLATE,spore-print-color_GREEN,spore-print-color_ORANGE,spore-print-color_PURPLE,spore-print-color_WHITE,spore-print-color_YELLOW,population_ABUNDANT,population_CLUSTERED,population_NUMEROUS,population_SCATTERED,population_SEVERAL,population_SOLITARY,habitat_GRASSES,habitat_LEAVES,habitat_MEADOWS,habitat_PATHS,habitat_URBAN,habitat_WASTE,habitat_WOODS
4933,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0
7476,0,1,0,1,1,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0
3490,1,1,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
2897,1,1,0,0,1,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
3129,1,1,0,0,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1


In [35]:
class_mapping = {'EDIBLE': 1, 'POISONOUS': 0}
y = y.map(class_mapping)



In [36]:
y.sample(n=5)

1236    1
6007    1
1340    1
5030    0
4054    0
Name: class, dtype: int64

In [37]:
X = X.values
y = y.values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

## sklearn model

In [38]:
model = MLPClassifier()

In [39]:
model.fit(X=X_train,y=y_train)

MLPClassifier()

In [40]:
model.score(X=X_test,y=y_test)

1.0

In [41]:
model.predict(X_test[101].reshape(1,-1)) 

array([0.], dtype=float32)

In [42]:
y_test[101]

tensor(0.)

## Своя модель

In [43]:
gdf = pd.read_csv("./mushroom/expanded.csv")
gdf.replace("?", pd.NA, inplace=True)
gdf['stalk-root'].fillna(gdf['stalk-root'].mode()[0], inplace=True)
u =gdf.apply(lambda x: x.unique())
X = gdf.drop(columns=['class'])
y = gdf['class']
cat_features = [x for x in X.columns]
ohe_features = [x for x in X.columns]
for col in cat_features:
    if (len(X[col].unique()) == 2):
        X[col] = (X[col] == X[col][0]).astype(int)
        ohe_features.remove(col)
ring_mapping = {'NONE':0, 'ONE': 1, 'TWO': 2}
X['ring-number'] = X['ring-number'].map(ring_mapping)
ohe_features.remove('ring-number')
X = pd.get_dummies(X, columns=ohe_features)
class_mapping = {'EDIBLE': 1, 'POISONOUS': 0}
y = y.map(class_mapping)





In [44]:
mushrooms = fetch_ucirepo(id=73)
df = mushrooms.data.features
target = mushrooms.data.targets['poisonous']

df['stalk-root'].fillna('m', inplace=True)
X = df

encoder = OneHotEncoder()
X_encoded = encoder.fit_transform(X).toarray()
y_encoded = target.apply(lambda x: 0 if x == 'p' else 1).values.reshape(-1, 1)



/tmp/ipykernel_16872/2363718341.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['stalk-root'].fillna('m', inplace=True)


In [45]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.2, random_state=65)



In [46]:
model = MLP([
    Linear(X_train.shape[1], 100),
    Sigmoid(),
    Linear(100, 1),
    Sigmoid()
], learning_rate=0.05, max_iter=100)
model.fit(X_train, y_train)

iteration: 0, loss: 20.266614852749907
iteration: 1, loss: 14.78439984595087
iteration: 2, loss: 11.151525208656643
iteration: 3, loss: 8.67749092637354
iteration: 4, loss: 6.943067880132247
iteration: 5, loss: 5.711173377932786
iteration: 6, loss: 4.832796701106078
iteration: 7, loss: 4.190232022572929
iteration: 8, loss: 3.7091214353071265
iteration: 9, loss: 3.3372642010383307
iteration: 10, loss: 3.0423071597627245
iteration: 11, loss: 2.803449312204278
iteration: 12, loss: 2.6061420962687887
iteration: 13, loss: 2.4403700296579256
iteration: 14, loss: 2.2993339919343305
iteration: 15, loss: 2.1792598230905718
iteration: 16, loss: 2.0755390891906584
iteration: 17, loss: 1.9833469116087685
iteration: 18, loss: 1.8996251780888176
iteration: 19, loss: 1.826770942511411
iteration: 20, loss: 1.7589323969783324
iteration: 21, loss: 1.6972275086757853
iteration: 22, loss: 1.6424829845152638
iteration: 23, loss: 1.5892746259625028
iteration: 24, loss: 1.5392386522207577
iteration: 25, loss

In [47]:
y_exp = y_test.T[0]
y_pred = model.predict(X_test).T[0]
model.score(y_exp, y_pred)

0.9709916909500949